In [1]:
import pandas as pd
import bz2

# BZ2 uzantılı dosyaları satır satır okuyan fonksiyon
def parse_bz2_data(file_path):
    data = []

    with bz2.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            label, text = line.strip().split(' ', 1)
            title, text = text.split(': ', 1)
            label = int(label.replace('__label__', ''))
            data.append((title.strip(), text.strip(), label))
    
    return pd.DataFrame(data, columns=['title', 'text', 'label'])

# 🔍 Dosya yolları (senin gönderdiğin dosya adları ile)
train_path = "train.ft.txt.bz2"
test_path = "test.ft.txt.bz2"

# ✅ Verileri oku
df_train = parse_bz2_data(train_path)
df_test = parse_bz2_data(test_path)

# 🔗 Tümünü birleştir
df = pd.concat([df_train, df_test], ignore_index=True)

# 🎯 Göz at
print(df.head())
print("\nVeri seti boyutu:", df.shape)

                                               title  \
0                     Stuning even for the non-gamer   
1              The best soundtrack ever to anything.   
2                                           Amazing!   
3                               Excellent Soundtrack   
4  Remember, Pull Your Jaw Off The Floor After He...   

                                                text  label  
0  This sound track was beautiful! It paints the ...      2  
1  I'm reading a lot of reviews saying that this ...      2  
2  This soundtrack is my favorite music of all ti...      2  
3  I truly like this soundtrack and I enjoy video...      2  
4  If you've played the game, you know how divine...      2  

Veri seti boyutu: (4000000, 3)


In [3]:
df.to_csv("amazon_dataset.csv")

NameError: name 'df' is not defined

In [4]:
import pandas as pd
import re, string
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset

# Sadece gerekli olan NLTK kaynaklarını indiriyoruz
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sagla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sagla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
# CSV dosyasını oku (aynı klasörde olduğunu varsayalım)
df = pd.read_csv('amazon_dataset.csv')
print("Veri kümesi yüklendi.")
print(df.head())

Veri kümesi yüklendi.
   Unnamed: 0                                              title  \
0           0                     Stuning even for the non-gamer   
1           1              The best soundtrack ever to anything.   
2           2                                           Amazing!   
3           3                               Excellent Soundtrack   
4           4  Remember, Pull Your Jaw Off The Floor After He...   

                                                text  label  
0  This sound track was beautiful! It paints the ...      2  
1  I'm reading a lot of reviews saying that this ...      2  
2  This soundtrack is my favorite music of all ti...      2  
3  I truly like this soundtrack and I enjoy video...      2  
4  If you've played the game, you know how divine...      2  


In [3]:
# title ve text alanlarını birleştir (araya boşluk koyarak)
df['review'] = df['title'] + ' ' + df['text']

# Kontrol et
print(df[['review', 'label']].head())

                                              review  label
0  Stuning even for the non-gamer This sound trac...      2
1  The best soundtrack ever to anything. I'm read...      2
2  Amazing! This soundtrack is my favorite music ...      2
3  Excellent Soundtrack I truly like this soundtr...      2
4  Remember, Pull Your Jaw Off The Floor After He...      2


In [6]:
import re

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply preprocessing
df['cleaned_review'] = df['review'].apply(preprocess_text)

In [7]:
def clean_text_basic(text):
    # Küçük harf yap
    text = text.lower()
    
    # HTML etiketlerini kaldır
    text = re.sub(r'<.*?>', '', text)
    
    # URL'leri kaldır
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Noktalama işaretlerini kaldır
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Emoji temizleme
    emoji_pattern = re.compile(
        "[" u"\U0001F600-\U0001F64F" u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF" u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0" "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    return text

In [5]:
# Sadece string olan ve NaN olmayan değerleri al
df = df[df['review'].apply(lambda x: isinstance(x, str))].copy()

In [10]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# POS bilgisi olmadan varsayılan isim (noun) kabul ederek sade lemmatization
def lemmatize_simple(text):
    words = text.split()  # burada tokenizer yerine split kullanıyoruz
    cleaned = [
        lemmatizer.lemmatize(word) for word in words
        if word.lower() not in stop_words and word.isalpha()
    ]
    return ' '.join(cleaned)

# Full temizlik + lemmatizasyon
def full_clean(text):
    text = clean_text_basic(text)
    text = lemmatize_simple(text)
    return text

# Uygula
df['cleaned_review'] = df['review'].apply(full_clean)

In [11]:
slang_dict = {"omg": "oh my god", "lol": "laugh out loud", "u": "you", "r": "are", "gr8": "great", "b4": "before", "asap": "as soon as possible", "btw": "", "fyi": "for your information", "idk": "i don't know", "imo": "in my opinion", "jk": "just kidding", "lmao": "laughing my ass off", "lmk": "let me know", "nvm": "nevermind", "np": "no problem", "rofl": "rolling on the floor laughing", "smh": "shaking my head", "tbh": "to be honest", "thx": "thanks", "ty": "thank you", "wth": "what the heck", "wtf": "what the f***", "yolo": "you only live once", "brb": "be right back", "gtg": "got to go", "btw": "by the way", "ttyl": "talk to you later", "ily": "I love you", "g2g": "got to go", "h8": "hate", "jk": "just kidding", "thx": "thanks", "ttyl": "talk to you later", "cya": "see you", "gg": "good game", "afk": "away from keyboard", "ez": "easy", "wb": "welcome back", "idc": "i don't care", "rn": "right now", "lmk": "let me know", "ikr": "i know right", "tmi": "too much information", "smh": "shaking my head", "w/e": "whatever", 
                 "bff": "best friends forever", "tfw": "that feeling when", "np": "no problem", "nvm": "nevermind", "fyi": "for your information", "cuz": "because", "gonna": "going to", "wanna": "want to", "gotta": "got to", "kinda": "kind of", "sorta": "sort of", "dunno": "don't know", "ain't": "is not", "gimme": "give me", "lemme": "let me", "im": "I'm", "hes": "he's", "shes": "she's", "theyre": "they're", "youre": "you're", "havent": "have not", "cant": "cannot",
                  "couldnt": "could not", "didnt": "did not", "doesnt": "does not", "dont": "do not", "isnt": "is not", "mightnt": "might not", "mustnt": "must not", "shouldnt": "should not", "wasnt": "was not", "werent": "were not", "wouldnt": "would not", "tho": "though", "thru": "through", "nite": "night", "neva": "never", "sum": "some", "dat": "that", "dis": "this", "dem": "them", "dey": "they", "em": "them", "whered": "where did", "whod": "who did", "whos": "who's", "yall": "you all", "dam": "damn", "hell": "hell", "pissed": "pissed", "fck": "fuck", "fuk": "fuck", "effing": "effing", "bs": "bullshit", "crap": "crap", "shitty": "shitty", "wtf": "what the fuck", "stfu": "shut the fuck up", "gtfo": "get the fuck out", "irl": "in real life", "roflmao": "rolling on the floor laughing my ass off", "smol": "small", "big oof": "huge mistake", "yeet": "throw something forcefully", "pog": "play of the game", "sus": "suspicious", "cap": "lie", "no cap": "no lie", "bet": "okay", "lit": "amazing", "fr": "for real", "bruh": "bro", "fam": "family",
                    "goat": "greatest of all time", "lowkey": "somewhat", "highkey": "very", "vibe": "mood", "drip": "fashionable", "slay": "do something well", "tea": "gossip", "sksksk": "excited reaction", "tf": "the fuck", "fomo": "fear of missing out", "tldr": "too long didn't read", "hmu": "hit me up", "wyd": "what you doing", "wym": "what you mean", "wdym": "what do you mean", "btw": "by the way", "imo": "in my opinion", "idc": "i don't care", 
                  "idgaf": "i don't give a f***", "frfr": "for real for real", "mf": "motherfucker", "rn": "right now", "lolz": "laughs", "hmu": "hit me up", "hbu": "how about you", "rn": "right now", "lmk": "let me know", "ikr": "i know right", "jk": "just kidding", "fr": "for real", "ffs": "for fuck's sake", "btw": "by the way", "nfs": "not for sale", "dm": "direct message", "af": "as fuck", "idk": "i don't know"}

def correct_slangs(text, slang_dict):
    for slang, replacement in slang_dict.items():
        text = re.sub(rf'\b{slang}\b', replacement, text)
    return text

# Slang düzeltmeyi orijinal review'e uygula (henüz temizlenmemiş)
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: correct_slangs(x.lower(), slang_dict))

In [12]:
df.to_csv("cleaned_amazon.csv")

In [1]:
import pandas as pd
df=pd.read_csv("cleaned_amazon.csv")

In [7]:
# NaN olanları temizle (her ihtimale karşı)
df = df.dropna(subset=['cleaned_review', 'label'])

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

# Etiketleri 2 → 1 (positive), 1 → 0 (negative) olacak şekilde dönüştür
df['label'] = df['label'].map({2: 1, 1: 0})

# Özellik ve etiketleri ayır
X = df['cleaned_review'].values
y = df['label'].values

# %80 train, %20 test
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Kontrol
print("Veri şekilleri:")
print(f"Train: {x_train.shape}, Test: {x_test.shape}")

Veri şekilleri:
Train: (3199792,), Test: (799949,)


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

# TfidfVectorizer (Binary + max 5000 feature)
vectorizer = TfidfVectorizer(max_features=5000)

x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Model eğitme ve değerlendirme fonksiyonu
def train_and_evaluate_model(model, model_name):
    print(f"\n🔹 {model_name}")
    model.fit(x_train_vec, y_train)
    preds = model.predict(x_test_vec)
    # Skorlar
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

MemoryError: 

In [ ]:
# Logistic Regression
train_and_evaluate_model(LogisticRegression(max_iter=1000), "Logistic Regression")


🔹 Logistic Regression
Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.89    401033
           1       0.88      0.89      0.89    398913

    accuracy                           0.89    799946
   macro avg       0.89      0.89      0.89    799946
weighted avg       0.89      0.89      0.89    799946

Logistic Regression - Accuracy on Training Set: 0.8865


In [ ]:
# Multinomial Naive Bayes
train_and_evaluate_model(MultinomialNB(), "Multinomial Naive Bayes")


🔹 Multinomial Naive Bayes
Multinomial Naive Bayes - Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.85      0.85    401033
           1       0.85      0.85      0.85    398913

    accuracy                           0.85    799946
   macro avg       0.85      0.85      0.85    799946
weighted avg       0.85      0.85      0.85    799946

Multinomial Naive Bayes - Accuracy on Training Set: 0.8482


In [ ]:
train_and_evaluate_model(LinearSVC(), "Linear SVC")

In [ ]:
# Random Forest
train_and_evaluate_model(RandomForestClassifier(n_estimators=100), "Random Forest")

In [ ]:
# SVM
train_and_evaluate_model(SVC(), "SVM")

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Tokenizer ayarları
max_words = 10000     # En sık kullanılan 10.000 kelime
max_len = 200         # Cümle uzunluğu sabitlenecek

# Tokenizer eğitimi
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

# Metinleri dizilere çevir
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Pad (doldurma)
x_train_pad = pad_sequences(x_train_seq, maxlen=max_len, padding='post', truncating='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len, padding='post', truncating='post')

# Etiketleri array yap
y_train_pad = np.array(y_train)
y_test_pad = np.array(y_test)

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

def create_model(model_type='rnn'):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_words, 64, input_length=max_len))
    
    if model_type == 'rnn':
        model.add(tf.keras.layers.SimpleRNN(64))
    elif model_type == 'lstm':
        model.add(tf.keras.layers.LSTM(64))
    elif model_type == 'cnn':
        model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
        model.add(tf.keras.layers.GlobalMaxPooling1D())
    
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_and_evaluate(model_type):
    print(f"\n📚 Training {model_type.upper()} model...")
    model = create_model(model_type)
    model.fit(x_train_pad, y_train, epochs=5, batch_size=128, validation_split=0.2, verbose=1)
    
    # Tahmin
    y_pred = (model.predict(x_test_pad) > 0.5).astype("int32")
    print(f"\n📊 {model_type.upper()} Classification Report:")
    print(classification_report(y_test, y_pred))


In [ ]:
for m in ['cnn', 'rnn', 'lstm']:
    train_and_evaluate(m)